In [1]:
options(digits=4, scipen=100)
a.matr <- c(1)

option.type <- list(call.option=1, put.option=2)

simulate.gbm <- function(S_0, r, sigma, dt, M, N){
    M<- M*length(S_0)
    S_0 <- rep(S_0, times=M)
    
    S <- matrix(S_0, nrow=M, ncol=N)
    sigma.sqrd <- sigma^2
    if((M%%2)!=0){
            stop('Number of Paths must be divisble by 2')
        }
    for(n in 2:N){
       
        half.point <- M/2
        
        std.normal <- numeric(M)
        std.normal[1:half.point] <- rnorm(half.point, mean=0, sd=1)
        std.normal[(half.point+1):M] <- -std.normal[1:half.point]
        S[,n] = S[,n-1] * exp((r-sigma.sqrd/2) * dt + sigma * std.normal * (dt^0.5))
    }
    
    return(S)
} 

bsm.price <- function(S_0, K, r, sigma, T){
    root.T <- sqrt(T)
    d1 <-(log(S_0/K) + (r-(sigma^2)/2) *T)/(sigma * sqrt(T))
    d2 <- d1 - sigma * root.T
    
    price <- K*exp(-r*T)*pnorm(-d2) - S_0 * pnorm(-d1) 
    return(price)
    
}

european.bermudan.asian <- function(prices.paths, K, r, T, type){
    aver <- rowMeans(prices.paths[,1:ncol(prices.paths)])
    res <- mean(pmax(aver-K, numeric(length(aver))) * exp(-r * T))
    
    return(res)
    
}

american.basis <- function(nrow, current.price, paths, payoffs, unadj.payoff,
                           K, valuation.time, additional.params){
    
    basis.order <- additional.params$basis.order
    X <- matrix(0,nrow=nrow, ncol=basis.order+1)
    X[,1] <- 1
    
    for(k in 1: basis.order){
        X[,k+1] <- current.price ^ k
    }
    
    
    return(X)
    
}

american.bermudan.asian.basis <- function(nrow, current.price, paths, payoffs,
                                          unadj.payoff, K, 
                                          valuation.time, additional.params){
    average.count <- additional.params$average.count
    
    average.start <- valuation.time-(average.count-1)
    
    window.average <- unadj.payoff[, valuation.time]
    
    X <- matrix(0,nrow=nrow, ncol=8)
    X[,1] <- 1
    X[,2] <- 1 - 2 *current.price + (current.price^2)/2
    X[,3] <- 1 - current.price
    
    X[,4] <- 1 - window.average
    X[,5] <- 1 - 2*window.average  + (window.average ^ 2)/2
    
    X[,6] <- X[,4] * X[,2]
    X[,7] <- X[,5] * X[,3]
    X[,8] <- X[,5] * X[,2]
    
    return(X)
    
}



generate.expected.payoffs <- function(paths, valuation.time,adj.payoffs,unadj.payoff,disc.rate, K,dt, config){
    current.price <- paths[,valuation.time]
    
    future.payoff <- as.matrix(adj.payoffs[,(valuation.time+1):ncol(adj.payoffs)])

    path_count <-length(current.price)

    X <- config$basis.func(nrow=path_count, current.price=current.price, paths=paths, 
                            payoffs=adj.payoffs, unadj.payoff=unadj.payoff, K=K,
                            valuation.time=valuation.time, additional.params=config$additional.params)
    
    current.payoff <- adj.payoffs[, valuation.time]
    
    #if(config$additional.params$type==option.type$put.option){}else{}
    
    X.filtered <- X[current.payoff>0,]
  
   
    usable.payoffs <- as.matrix(future.payoff[current.payoff>0,])
    
    disc.factor <- as.matrix(exp(-disc.rate * 1:ncol(future.payoff) * dt))
    tryCatch({
        y <- usable.payoffs %*% disc.factor
    }, error=function(cond){
        print(dim(usable.payoffs))
       
        
        print(dim(disc.factor))
        stop('usable.payoffs and disc.factor were not conformable')
    })
    

    transpose.x <- t(X.filtered)
    
    if(config$additional.params$should.fix.betas){
        betas <- config$additional.params$fixed.betas[valuation.time ,]
      
    }else{
        tryCatch({
            betas <- solve(transpose.x %*% X.filtered, tol=1e-32) %*% transpose.x %*% y
            },error=function(err){
                print(err)
                print(valuation.time)
                assign("a.matr", transpose.x %*% X.filtered, envir = .GlobalEnv)
                print(transpose.x %*% X.filtered)
                stop('Error occured in inverting matrix')
        })
    }
    # error handling for when inversion of transpose.x %*% X.filtered fails
    
    
    #print(betas)
    return (list(expected=X %*% betas, betas=as.vector(betas)))
}

compare.continuation <- function(expected.payoff, payoff.matrix, compare.col.num){
    cols <- ncol(payoff.matrix)
    
    # set current payoff=0 if expected is greator, investor chooses continuation
    payoff.matrix[expected.payoff > payoff.matrix[,compare.col.num], compare.col.num] <- 0
    # set future payoffs to 0 if current payoff is deemed better than expected
    payoff.matrix[payoff.matrix[,compare.col.num] > 0, (compare.col.num+1) : cols] <- 0
    return (payoff.matrix)
}

american.bermuda.payoff <- function(paths, K, path.count, time_steps,additional.params ){
    # assumes is call
    # periods before the minimum count remain as 0
    
    t <- time_steps
    payoff.matrix <- matrix(0, ncol=time_steps, nrow=path.count)
    average.count <- additional.params$average.count
    
    if(is.null(average.count)){
        stop(' Average count not provided')
    }
    
    while(t>average.count){
        start.time<-(t-(average.count-1))
        payoff.matrix[,t] <- rowMeans( paths[,1:t])
        
        t <- t-1
        
    }
    
    payoff.matrix[,average.count] <- additional.params$initial.average
    
    res <- list(un.adj=payoff.matrix, adj=payoff.matrix-K)
    return(res)
}

american.payoff <- function(paths, K, path.count, time_steps, additional.params){
    payoff.matrix <- K - paths
    res <- list(un.adj=NULL, adj=payoff.matrix)
    return(res)
}

cashflow.matrix <- function(paths, K, r, dt, config){
    
    time_steps <- ncol(paths)
    path.count <- nrow(paths)
    
    valuation.end.time <- config$additional.params$valuation.end.time
    
    if(is.null(valuation.end.time)){
        stop('Missing valuation end time')
    }
    
    result <- config$payoff.func(paths, K, path.count, time_steps, config$additional.params)
    unadj.payoff <-  result$un.adj
    payoff.matrix <- result$adj
    
    for(t in 1: time_steps){
        payoff.matrix[,t] <- pmax(payoff.matrix[,t], numeric(path.count))
    }
    
    valuation.start.time <- time_steps-1
    betas.store <- NULL
    regression.count <-  valuation.start.time 
    
    for(t in valuation.start.time:valuation.end.time){
        
        fit <- generate.expected.payoffs(paths, valuation.time=t,adj.payoffs=payoff.matrix, 
                                              unadj.payoff = unadj.payoff,
                                              disc.rate=r, K=K, dt=dt, config=config)
        expected <- fit$expected
        
        # store the betas
        if(is.null(betas.store)){
          
            betas.store <- matrix(0, nrow=regression.count, ncol=length(fit$betas))
            betas.store[t, ] <- fit$betas
        }
        
        
        betas.store[t, ] <- fit$betas
        payoff.matrix <- compare.continuation(expected,payoff.matrix,t)
    
    }
    payoff.beta <- list(payoff=payoff.matrix[,valuation.end.time:time_steps], betas=betas.store)
    return(payoff.beta)
}


least.sq.price <- function (paths, K,r,dt, ...){
    config <- list(...)
    cf.model <- cashflow.matrix(paths, K, r, dt, config)
    cf <- cf.model$payoff
    disc.factor <- as.matrix(exp(-r * 1:ncol(cf) *dt))
    discounted.prices <- cf %*% disc.factor
    std.error <- sqrt(var(discounted.prices)) * 1.96 * (1/sqrt(length(discounted.prices)))
    res <-  list(price=mean(discounted.prices), std.error=std.error[1], betas=cf.model$betas)
    return(res)
}

# Test on longstaff's sample


In [9]:
longstaff.sample <- matrix(1, nrow=8, ncol=4)
longstaff.sample[1,2:4] <- c(1.09,1.08,1.34)
longstaff.sample[2,2:4] <- c(1.16,1.26,1.54)
longstaff.sample[3,2:4] <- c(1.22,1.07,1.03)
longstaff.sample[4,2:4] <- c(0.93,0.97,0.92)
longstaff.sample[5,2:4] <- c(1.11,1.56,1.52)
longstaff.sample[6,2:4] <- c(0.76,0.77,0.9)
longstaff.sample[7,2:4] <- c(0.92,0.84,1.01)
longstaff.sample[8,2:4] <- c(0.88,1.22,1.34)
sample.params <- list(valuation.end.time=2, type=option.type$put.option, 
                      basis.order=2, should.fix.betas=FALSE, fixed.betas=NULL)
model <- least.sq.price(longstaff.sample, K=1.1, r=0.06, dt=1, basis.func=american.basis, 
               payoff.func=american.payoff, additional.params=sample.params)
model$price[1]
model$std.error[1]
#model$betas[1:2,]

[1] 0.1144

[1] 0.08219

# American Put Option

### generate table 1

In [3]:
generate.table.1 <- function(initial.prices, sigmas, years.vector,r, K, M, N, 
                             basis.func, payoff.func, additional.params){
    result.length <- length(initial.prices) *length(sigmas) * length(years.vector)
    result <- matrix(0, nrow=result.length, ncol=7)
    colnames(result) <- c('S', 'sigma', 'T', 'LSM Price',"Std. Error", "Closed Form", "Early Exercise Value")
    
    dt <- 1/N
    
    i <- 1
    for(sigma in sigmas){
        for(time in years.vector){
            prices <- simulate.gbm(initial.prices, r, sigma, dt=dt, M=M, N=N*time)
            for(price in initial.prices){
                LSM.price <- least.sq.price(prices[prices[,1]==price,], K, r, dt=dt, basis.func=basis.func, 
                                payoff.func=payoff.func, additional.params=additional.params)
                closed.form <- bsm.price(S_0 = price, K = K, r = r, sigma = sigma,T = time)
                early.exercise <- LSM.price$price - closed.form
                result[i,] <- c(price, sigma, time, LSM.price$price, LSM.price$std.error, closed.form, early.exercise)
                i <- i+1
                
            }
            
        }
            
    }
    return(result[order(result[,1]),])
}
additional.params <- list(valuation.end.time=2, type=option.type$put.option, basis.order=3, should.fix.betas=FALSE, fixed.betas=NULL)

table1 <- generate.table.1(c(36,38, 42), c(0.2, 0.4), c(1,2), r=0.06, K=40, M=10000, N=100, basis.func=american.basis, 
               payoff.func=american.payoff, additional.params=additional.params)
table1

S,sigma,T,LSM Price,Std. Error,Closed Form,Early Exercise Value
36,0.2,1,4.492,0.05411,3.790,0.7027
36,0.2,2,4.849,0.06974,3.605,1.2439
36,0.4,1,7.117,0.11854,6.284,0.8330
36,0.4,2,8.567,0.13889,6.541,2.0260
38,0.2,1,3.238,0.05837,2.792,0.4458
38,0.2,2,3.762,0.06788,2.827,0.9354
38,0.4,1,6.128,0.11554,5.377,0.7514
38,0.4,2,7.655,0.13681,5.754,1.9016
42,0.2,1,1.613,0.04646,1.407,0.2067
42,0.2,2,2.190,0.05870,1.683,0.5072


# Convergence test

### Table 2

In [5]:
generate.table2 <- function(initial.prices, K, r, sigmas, seed.count, 
                            M, N, basis.func, payoff.func, additional.params,
                            T=1){
    dt = 1/N
    result.length <- length(initial.prices) * length(sigmas) * length(T) * seed.count
    print(result.length)
    result <- matrix(0, nrow=result.length, ncol=8)
    colnames(result) <- c('S', 'sigma', 'T', 'LSM Price (in sample)',"Std. Error (in sample)", 
                          "LSM Price (out of sample)", "Std. Error(out of sample)", "Difference")
    i <- 1
    for(sample in 1:seed.count){
        for(sigma in sigmas){
           sample.prices <- simulate.gbm(S_0 = initial.prices, r = r, sigma = sigma, M = M, N=N,dt = dt)
           sample.prices.fixed <- simulate.gbm(S_0 = initial.prices, r = r, sigma = sigma, M = M, N=N,dt = dt)
            for(price in initial.prices){
                simulated.model <- least.sq.price(paths = sample.prices[sample.prices[,1]==price,], 
                                            K = K, r = r, dt=dt,
                                            basis.func=basis.func, 
                                            payoff.func=payoff.func, 
                                            additional.params=additional.params)
                # fix betas
                add.params.fix <- additional.params
                add.params.fix$should.fix.betas=TRUE
                add.params.fix$fixed.betas=simulated.model$betas
               
                simulated.model.fixed <- least.sq.price(paths = sample.prices.fixed[sample.prices.fixed[,1]==price,], 
                                            K = K, r = r, dt=dt,
                                            basis.func=basis.func, 
                                            payoff.func=payoff.func, 
                                            additional.params=add.params.fix)
                
                row <- c(price, sigma, T, simulated.model$price, simulated.model$std.error, 
                        simulated.model.fixed$price, simulated.model.fixed$std.error,
                        simulated.model$price-simulated.model.fixed$price)
                result[i,] <- row
                
                i<- i+1
                
            }   
        }
          
    }
  return(result[order(result[,1]),])
}
generate.table2(c(36, 44), r=0.06, K=40, sigmas=c(0.2), seed.count=5, T=1, M=100000, N=50, 
                basis.func=american.basis,
                payoff.func=american.payoff,
                additional.params=additional.params)

[1] 10


S,sigma,T,LSM Price (in sample),Std. Error (in sample),LSM Price (out of sample),Std. Error(out of sample),Difference
36,0.2,1,4.464,0.01770,4.466,0.01769,-0.0022202
36,0.2,1,4.464,0.01823,4.475,0.01819,-0.0107642
36,0.2,1,4.468,0.01791,4.470,0.01791,-0.0015463
36,0.2,1,4.462,0.01810,4.462,0.01810,-0.0004288
36,0.2,1,4.473,0.01793,4.471,0.01791,0.0021270
44,0.2,1,1.103,0.01269,1.090,0.01261,0.0133433
44,0.2,1,1.090,0.01265,1.090,0.01267,-0.0003144
44,0.2,1,1.097,0.01267,1.097,0.01265,0.0005074
44,0.2,1,1.090,0.01261,1.094,0.01261,-0.0042069
44,0.2,1,1.096,0.01264,1.090,0.01264,0.0055532


# American Asian Bermudan Call option

In [8]:
look.back <- 25
additional.params.bermudan <- list(valuation.end.time=look.back+1, average.count=look.back, 
                                   type=option.type$call.option, initial.average=110,
                                   should.fix.betas=FALSE, fixed.betas=NULL)

generate.table3 <- function(initial.prices, K, r, sigma, initial.averages, M, N, T, look.back, type=option.type$call.option){
    dt <-1/N
    result.length <- length(initial.prices) * length(initial.averages)
    result <- matrix(0, nrow=result.length, ncol=6)
    colnames(result) <- c('S', 'sigma', 'LSM Price',"Std. Error", "European Asian", "Early Exercise Value")
    
    i <- 1
    sample.prices <- simulate.gbm(S_0 =initial.prices, r=r,sigma=sigma, dt=dt, 
                                       M =M, N=N *T)
    for(price in initial.prices){
         sample <- sample.prices[sample.prices[,1]==price,]
        
        for(average in initial.averages){
            additional.params <- list(valuation.end.time=look.back+1, average.count=look.back, type=type, 
                                      initial.average=average, should.fix.betas=FALSE, fixed.betas=NULL)

            option.price <- least.sq.price(sample, K=K, r=r, 
                                           dt=dt, basis.func=american.bermudan.asian.basis, 
                                           payoff.func=american.bermuda.payoff, 
                                           additional.params=additional.params)
            european.price <- european.bermudan.asian(sample, K, r, T, type)
            row <- c(price, sigma, option.price$price, option.price$std.error, european.price, option.price$price-european.price)
            result[i,] <- row
            i <- i+1
            
            
        }
        
    }
    return(result)
}

generate.table3(c(90,80,100), K=100, r=0.06, sigma=0.2, initial.averages=c(90), M=50000, N=100, T=2, look.back=look.back)

S,sigma,LSM Price,Std. Error,European Asian,Early Exercise Value
90,0.2,4.213,0.07059,3.975,0.23826
80,0.2,1.225,0.03701,1.185,0.03992
100,0.2,10.055,0.10607,9.167,0.88797
